In [5]:
import pymysql

mydb = pymysql.connect(
    host = 'localhost',
    user = 'root',
    password = 'QWERTY#123#'
)


# mycursor = mydb.cursor()
# mycursor.execute('create database foodordering_system')
# mycursor.execute('show databases')
# for x in mycursor:
#     print(x)


mydb = pymysql.connect(
    host = 'localhost',
    user = 'root',
    password = 'QWERTY#123#',
    database = 'foodordering_system'
)
mycursor = mydb.cursor()

# mycursor.execute('''
#     CREATE TABLE customerrecords1(
#         OrderID VARCHAR(255) PRIMARY KEY NOT NULL,
#         Name VARCHAR(255) NOT NULL,
#         Email TEXT NOT NULL,
#         PhoneNum VARCHAR(50) NOT NULL,
#         Address TEXT NOT NULL,
#         PaymentMethod VARCHAR(50) NOT NULL
#     )
# ''')

mycursor.execute('SHOW COLUMNS FROM customerrecords')
for x in mycursor:
    print(x)
    


('OrderID', 'int', 'NO', 'PRI', None, '')
('Name', 'varchar(255)', 'NO', '', None, '')
('Email', 'text', 'NO', '', None, '')
('PhoneNum', 'varchar(50)', 'NO', '', None, '')
('Address', 'text', 'NO', '', None, '')
('PaymentMethod', 'varchar(50)', 'NO', '', None, '')


In [16]:
import string
import random
from datetime import datetime

order_dict = {}
totalprice = 0.0

def get_non_empty_input(prompt):
    while True:
        user_input = input(prompt)
        if user_input.strip(): 
            return user_input

def generate_order_id():
    alength = 4
    numeric_chars = string.digits
    return ''.join(random.choice(numeric_chars) for _ in range(alength))

def customerinfo():
    OrderID = generate_order_id()
    print(f'Your Order ID is {OrderID}')
    Name = get_non_empty_input('Provide your Full Name: ')
    Email = get_non_empty_input('Provide your Email Address: ')
    PhoneNum = get_non_empty_input('Provide your Phone Number: ')
    Address = get_non_empty_input('Kindly provide your Residential address: ')
    PaymentMethod = get_non_empty_input('Provide Mode of Payment(Cash/Card/Transfer): ')

    query = 'INSERT INTO customerrecords(OrderID,Name,Email,PhoneNum,Address,PaymentMethod) VALUES(%s, %s, %s, %s, %s, %s)'
    mycursor.execute(query, (OrderID, Name, Email, PhoneNum, Address, PaymentMethod))
    mydb.commit()
    print('Customer order details successfully saved...')
    
def updateCustomerDetails():
    OrderID = (get_non_empty_input('Enter OrderID to be updated: '))
    column = get_non_empty_input('Enter the column to be updated(Name,Email,Gender,PhoneNum,Address,PaymentMethod): ')
    newval = get_non_empty_input('Enter the new value: ')    
    query = f'UPDATE customerrecords SET {column} = %s WHERE OrderID = %s'
    mycursor.execute(query, (newval, OrderID))
    mydb.commit()
    print('Customer details updated successfully...')
    
def displayAll():
    mycursor.execute('SELECT * FROM customerrecords')
    for x in mycursor:
        print(x)

def order():
    global order_dict, totalprice
    menu = {
        'Sharwama': 1500.0,
        'Sandwich': 7500.0,
        'Salad': 4990.0,
        'Fries': 3990.0,
        'Burger': 8000,
    }    
    
    while True:
        OrderID = input('Enter Order ID: ')
        if OrderID.lower() == 'done':
            break  # Exit the loop if the user enters 'done'
        
        query = 'SELECT * FROM customerrecords WHERE OrderID = %s'
        mycursor.execute(query, (OrderID,))
        result = mycursor.fetchone()
        
        if result:
            print('\nMENU')
            for item, price in menu.items():
                print(f'{item.capitalize()}: N{price:.2f}')
            user_choice = input('Enter the food item you want to order (or done to finish): ')     
            
            if user_choice.lower() == 'done':
                break  # Exit the loop if the user enters 'done'
            
            if user_choice in menu:
                quantity = int(input(f'How many {user_choice}s do you want to order? '))
                order_dict[user_choice] = quantity
                totalprice += menu[user_choice] * quantity
                print(f'Total Price: N{totalprice:.2f}')
            else:
                print('Invalid choice, choose from the menu.')
        else:
            print('Customer\'s Order ID not found!')

    return OrderID, order, totalprice

def check():
    OrderID = (input('Enter Order ID: '))
    query = 'SELECT * FROM customerrecords WHERE OrderID = %s'
    mycursor.execute(query, (OrderID,))
    result = mycursor.fetchall()
    
    if len(result) == 0:
        print('Record not found!')
    else:
        for x in result:
            print(x)
            
def receipt():
    OrderID = (input('Enter Order ID: '))
    query = 'SELECT * FROM customerrecords WHERE OrderID = %s'
    mycursor.execute(query, (OrderID,))
    result = mycursor.fetchone()
    
    if not result:
        print('Customer\'s Order ID not found!')
        return
    
    print('Customer Details.')
    print(f'''
OrderID:{result[0]}
Name: {result[1]}
Email: {result[2]}
PhoneNum: {result[3]}
Address: {result[4]}
PaymentMethod: {result[5]}
''')
    
    print('\nRECEIPT')
    current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f'\nDate and Time: {current_datetime}')
    
    for item, quantity in order_dict.items():
        print(f'{item.capitalize()}: {quantity} x ${menu[item]:.2f} = ${menu[item] * quantity:.2f}')
        
    print(f'Total Price is ${totalprice}.\nProceed to Payment.')
    
def deleteorder():
    OrderID = (input('Enter OrderID to be deleted: '))
    ask = input('Are you sure you want to delete this order(Yes/No)? ')
    if ask.lower() == 'yes':
        query = 'DELETE FROM customerrecords WHERE OrderID = %s'
        mycursor.execute(query, (OrderID,))
        mydb.commit()
        print('Customer\'s order deleted successfully...')
    else:
        print('Deletion canceled.')


def main_admin():
    operation = True
    passid = 'admin'
    password = get_non_empty_input('Enter your password: ')
    if passid == password:
        while operation:
            print('Welcome to Mara\'s Food Depot!')
            print('Operation[Display, Check, Delete, Logout]')
            option = input('Select Operation: ')
            if option == 'Display'.lower():
                displayAll() 
            elif option == 'Check'.lower():
                receipt()
            elif option == 'Delete'.lower():
                deleteorder()
            elif option == 'Logout'.lower():
                operation = False
                print('You are Logged out.')
                break                
    else:
        print('Incorrect Password!')

def main_customer():
    operation = True
    while operation:
        print('Welcome to Mara\'s Food Depot!')
        print('Operation[Details, Order, Receipt, Update, Delete, Logout]')
        option = input('Select Operation: ')
        if option == 'Details'.lower():
            customerinfo() 
        elif option == 'Order'.lower():
            order()
        elif option == 'Receipt'.lower():
            receipt()
        elif option == 'Update'.lower():
            updateCustomerDetails()
        elif option == 'Delete'.lower():
            deleteorder()
        elif option == 'Logout'.lower():
            operation = False
            print('You are Logged out.')
            break

In [13]:
main_admin()

Enter your password: admin
Welcome to Mara's Food Depot!
Operation[Display, Check, Delete, Logout]
Select Operation: display
(2648, 'Mara Reuben', 'amara@gmail.com', '09013265275', '222 Bakori Road, Kubwa', 'Transfer')
(7314, 'Amara Beans', 'beans@gmail.com', '09012365276', '23rd Road, Abuja', 'Transfer')
(8376, 'Simeon Python', 'python@gmail.com', '090876543', '222 Pronet Mall, kubwa', 'Cash')
Welcome to Mara's Food Depot!
Operation[Display, Check, Delete, Logout]
Select Operation: logout
You are Logged out.


In [14]:
main_customer()

Welcome to Mara's Food Depot!
Operation[Details, Order, Receipt, Update, Delete, Logout]
Select Operation: receipt
Enter Order ID: 8376
Customer Details.

OrderID:8376
Name: Simeon Python
Email: python@gmail.com
PhoneNum: 090876543
Address: 222 Pronet Mall, kubwa
PaymentMethod: Cash


RECEIPT

Date and Time: 2024-02-17 16:35:45
Total Price is $0.0.
Proceed to Payment.
Welcome to Mara's Food Depot!
Operation[Details, Order, Receipt, Update, Delete, Logout]
Select Operation: logout
You are Logged out.


In [15]:
main_customer()

Welcome to Mara's Food Depot!
Operation[Details, Order, Receipt, Update, Delete, Logout]
Select Operation: order
Enter Order ID: 2648

MENU
Sharwama: N1500.00
Sandwich: N7500.00
Salad: N4990.00
Fries: N3990.00
Burger: N8000.00
Enter the food item you want to order (or done to finish): Sharwama
How many Sharwamas do you want to order? 5
Total Price: N7500.00
Enter Order ID: done
Welcome to Mara's Food Depot!
Operation[Details, Order, Receipt, Update, Delete, Logout]
Select Operation: receipt
Enter Order ID: 2648
Customer Details.

OrderID:2648
Name: Mara Reuben
Email: amara@gmail.com
PhoneNum: 09013265275
Address: 222 Bakori Road, Kubwa
PaymentMethod: Transfer


RECEIPT

Date and Time: 2024-02-17 17:44:08


NameError: name 'menu' is not defined

In [28]:
import pymysql

mydb = pymysql.connect(
    host = 'localhost',
    user = 'root',
    password = 'QWERTY#123#'
)


# mycursor = mydb.cursor()
# mycursor.execute('create database foodordering_system')
# mycursor.execute('show databases')
# for x in mycursor:
#     print(x)


mydb = pymysql.connect(
    host = 'localhost',
    user = 'root',
    password = 'QWERTY#123#',
    database = 'foodordering_system'
)
mycursor = mydb.cursor()

# mycursor.execute('''
#     CREATE TABLE customerrecords1(
#         OrderID VARCHAR(255) PRIMARY KEY NOT NULL,
#         Name VARCHAR(255) NOT NULL,
#         Email TEXT NOT NULL,
#         PhoneNum VARCHAR(50) NOT NULL,
#         Address TEXT NOT NULL,
#         PaymentMethod VARCHAR(50) NOT NULL
#     )
# ''')

mycursor.execute('SHOW TABLE FROM foodordering_sytem')
for x in mycursor:
    print(x)


ProgrammingError: (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'FROM foodordering_sytem' at line 1")